In [61]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [62]:
from model.DogCatNet import DogCatVGGNet

In [63]:
transforms = transforms.Compose([
    transforms.RandomResizedCrop(226),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
batch_size = 64
train_dataset = datasets.ImageFolder("./dataset/catsdogs/train/", transforms)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = datasets.ImageFolder("./dataset/catsdogs/test/", transforms)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [64]:
model = DogCatVGGNet()
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [65]:
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        # 获得一个批次的数据和标签
        inputs, target = data
        optimizer.zero_grad()
        # 获得模型预测结果
        outputs = model(inputs)
        # 交叉熵代价函数outputs(64,10),target（64）
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0
 
 
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1) # dim = 1 列是第0个维度，行是第1个维度
            total += labels.size(0)
            correct += (predicted == labels).sum().item() # 张量之间的比较运算
    return correct/total

In [66]:
if __name__ == '__main__':
    accuracy_max = 0
    accuracy = 0
    accuracys = []
    for epoch in range(1000):
        train(epoch)
        accuracy = test()
        if(accuracy_max < accuracy):
            accuracy_max = accuracy
            print("save: train_accuracy (%d,%d) epoch:%d"%(accuracy* 100,accuracy_max * 100,epoch))
            torch.save(model.state_dict(), "./save/dogcat.pt")
        accuracys.append(accuracy)
    print("accuracy (%d,%d)"%(accuracy* 100,accuracy_max * 100))
    plt.plot(accuracys)

<built-in method size of Tensor object at 0x0000026F23B9CDB0>
Hello


RuntimeError: mat1 and mat2 shapes cannot be multiplied (25088x64 and 25088x4096)